# Vexpresso Walkthrough

## In this walkthrough, we will be using a small dataset of Pokemon statistics to showcase vexpresso's capabilities

### Imports

In [1]:
import json
import pandas as pd
from langchain.embeddings import HuggingFaceHubEmbeddings, HuggingFaceEmbeddings
import vexpresso
from vexpresso.retriever import NumpyRetriever
import numpy as np
import daft

/home/shyam/miniconda3/envs/py39/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Create Collection
### This may take some time because we are embedding 809 entries

In [2]:
with open("./data/pokedex_processed.json", 'r') as f:
    stuff = json.load(f)

df = pd.DataFrame(stuff)

### Pokemon data

In [3]:
df.head(10)

,id,type,info,profile,name,description
0,1,"['Grass', 'Poison']","{'HP': 45, 'Attack': 49, 'Defense': 49, 'Sp. A...","{'height': '0.7 m', 'weight': '6.9 kg', 'egg':...",Bulbasaur,Seed Pokémon: Bulbasaur can be seen napping in...
1,2,"['Grass', 'Poison']","{'HP': 60, 'Attack': 62, 'Defense': 63, 'Sp. A...","{'height': '1 m', 'weight': '13 kg', 'egg': ['...",Ivysaur,Seed Pokémon: There is a bud on this Pokémon’s...
2,3,"['Grass', 'Poison']","{'HP': 80, 'Attack': 82, 'Defense': 83, 'Sp. A...","{'height': '2 m', 'weight': '100 kg', 'egg': [...",Venusaur,Seed Pokémon: There is a large flower on Venus...
3,4,['Fire'],"{'HP': 39, 'Attack': 52, 'Defense': 43, 'Sp. A...","{'height': '0.6 m', 'weight': '8.5 kg', 'egg':...",Charmander,Lizard Pokémon: The flame that burns at the ti...
4,5,['Fire'],"{'HP': 58, 'Attack': 64, 'Defense': 58, 'Sp. A...","{'height': '1.1 m', 'weight': '19 kg', 'egg': ...",Charmeleon,Flame Pokémon: Charmeleon mercilessly destroys...
5,6,"['Fire', 'Flying']","{'HP': 78, 'Attack': 84, 'Defense': 78, 'Sp. A...","{'height': '1.7 m', 'weight': '90.5 kg', 'egg'...",Charizard,Flame Pokémon: Charizard flies around the sky ...
6,7,['Water'],"{'HP': 44, 'Attack': 48, 'Defense': 65, 'Sp. A...","{'height': '0.5 m', 'weight': '9 kg', 'egg': [...",Squirtle,Tiny Turtle Pokémon: Squirtle’s shell is not m...
7,8,['Water'],"{'HP': 59, 'Attack': 63, 'Defense': 80, 'Sp. A...","{'height': '1 m', 'weight': '22.5 kg', 'egg': ...",Wartortle,Turtle Pokémon: Its tail is large and covered ...
8,9,['Water'],"{'HP': 79, 'Attack': 83, 'Defense': 100, 'Sp. ...","{'height': '1.6 m', 'weight': '85.5 kg', 'egg'...",Blastoise,Shellfish Pokémon: Blastoise has water spouts ...
9,10,['Bug'],"{'HP': 45, 'Attack': 30, 'Defense': 35, 'Sp. A...","{'height': '0.3 m', 'weight': '2.9 kg', 'egg':...",Caterpie,Worm Pokémon: Its body is soft and weak. In na...


In [4]:
# embeddings = HuggingFaceEmbeddings()
# def embedding_fn(content):
#     return np.array(embeddings.embed_documents(content))

collection = vexpresso.create(
    content="description",
    metadata=df,
    embedding_fn=HuggingFaceEmbeddings(),
    retriever=NumpyRetriever()
)

2023-06-01 00:29:50.298 | INFO     | daft.context:runner:88 - Using PyRunner


In [5]:
collection.show(5)

descriptionUtf8,idInt64,typeUtf8,"infoStruct[Attack: Int64, Defense: Int64, HP: Int64, Sp. Attack: Int64, Sp. Defense: Int64, Speed: Int64]","profileStruct[ability: List[List[Utf8]], egg: List[Utf8], gender: Utf8, height: Utf8, weight: Utf8]",nameUtf8,embeddings_descriptionPython,modal_map_indexInt64
Seed Pokémon: Bulbasaur can be seen napping in bright sun...,1,"['Grass', 'Poison']","{'Attack': 49, 'Defense': 49, 'HP': 45, 'Sp. Attack': 65,...","{'ability': [['Overgrow', 'false'], ['Chlorophyll', 'true...",Bulbasaur,"<np.ndarrayshape=(768,)dtype=float64>",0
Seed Pokémon: There is a bud on this Pokémon’s back. To s...,2,"['Grass', 'Poison']","{'Attack': 62, 'Defense': 63, 'HP': 60, 'Sp. Attack': 80,...","{'ability': [['Overgrow', 'false'], ['Chlorophyll', 'true...",Ivysaur,"<np.ndarrayshape=(768,)dtype=float64>",1
Seed Pokémon: There is a large flower on Venusaur’s back....,3,"['Grass', 'Poison']","{'Attack': 82, 'Defense': 83, 'HP': 80, 'Sp. Attack': 100...","{'ability': [['Overgrow', 'false'], ['Chlorophyll', 'true...",Venusaur,"<np.ndarrayshape=(768,)dtype=float64>",2
Lizard Pokémon: The flame that burns at the tip of its ta...,4,['Fire'],"{'Attack': 52, 'Defense': 43, 'HP': 39, 'Sp. Attack': 60,...","{'ability': [['Blaze', 'false'], ['Solar Power', 'true']]...",Charmander,"<np.ndarrayshape=(768,)dtype=float64>",3
Flame Pokémon: Charmeleon mercilessly destroys its foes u...,5,['Fire'],"{'Attack': 64, 'Defense': 58, 'HP': 58, 'Sp. Attack': 80,...","{'ability': [['Blaze', 'false'], ['Solar Power', 'true']]...",Charmeleon,"<np.ndarrayshape=(768,)dtype=float64>",4


## Querying the collection's embeddings, lazy execution

In [6]:
sleepy_pokemon = collection.query(query={"description":["Loves to sleep"]}, k=10)

### This allows for query chaining

In [7]:
sleepy_pokemon = sleepy_pokemon.query(query={"description":["Water body"]}, k=5)

In [8]:
executed = sleepy_pokemon.execute()

#### Show daft DataFrame

In [9]:
executed.show(5)

descriptionUtf8,idInt64,typeUtf8,"infoStruct[Attack: Int64, Defense: Int64, HP: Int64, Sp. Attack: Int64, Sp. Defense: Int64, Speed: Int64]","profileStruct[ability: List[List[Utf8]], egg: List[Utf8], gender: Utf8, height: Utf8, weight: Utf8]",nameUtf8,embeddings_descriptionPython,modal_map_indexInt64,scoreFloat64
Soft Tissue Pokémon: Its body is mostly water. If it drie...,704,['Dragon'],"{'Attack': 50, 'Defense': 35, 'HP': 45, 'Sp. Attack': 55,...","{'ability': [['Sap Sipper', 'false'], ['Hydration', 'fals...",Goomy,"<np.ndarrayshape=(768,)dtype=float64>",703,0.310911
Tiny Robin Pokémon: Its body is always warm. Trainers who...,661,"['Normal', 'Flying']","{'Attack': 50, 'Defense': 43, 'HP': 45, 'Sp. Attack': 40,...","{'ability': [['Big Pecks', 'false'], ['Gale Wings', 'true...",Fletchling,"<np.ndarrayshape=(768,)dtype=float64>",660,0.122607
Flycatcher Pokémon: Weepinbell has a large hook on its re...,70,"['Grass', 'Poison']","{'Attack': 90, 'Defense': 50, 'HP': 65, 'Sp. Attack': 85,...","{'ability': [['Chlorophyll', 'false'], ['Gluttony', 'true...",Weepinbell,"<np.ndarrayshape=(768,)dtype=float64>",69,0.121085
Drowsing Pokémon: It remains asleep from birth to death a...,775,['Normal'],"{'Attack': 115, 'Defense': 65, 'HP': 65, 'Sp. Attack': 75...","{'ability': [['Comatose', 'false']], 'egg': ['Field'], 'g...",Komala,"<np.ndarrayshape=(768,)dtype=float64>",774,0.104583
Pitch-Black Pokémon: It can lull people to sleep and make...,491,['Dark'],"{'Attack': 90, 'Defense': 90, 'HP': 70, 'Sp. Attack': 135...","{'ability': [['Bad Dreams', 'false']], 'egg': ['Undiscove...",Darkrai,"<np.ndarrayshape=(768,)dtype=float64>",490,0.0887914


#### Show pandas

In [10]:
executed.to_pandas()

,description,id,type,info,profile,name,embeddings_description,modal_map_index,score
0,Soft Tissue Pokémon: Its body is mostly water....,704,['Dragon'],"{'Attack': 50, 'Defense': 35, 'HP': 45, 'Sp. A...","{'ability': [['Sap Sipper', 'false'], ['Hydrat...",Goomy,"[0.016569413244724274, -0.023747211322188377, ...",703,0.310911
1,Tiny Robin Pokémon: Its body is always warm. T...,661,"['Normal', 'Flying']","{'Attack': 50, 'Defense': 43, 'HP': 45, 'Sp. A...","{'ability': [['Big Pecks', 'false'], ['Gale Wi...",Fletchling,"[0.01935332827270031, -0.08202998340129852, 0....",660,0.122607
2,Flycatcher Pokémon: Weepinbell has a large hoo...,70,"['Grass', 'Poison']","{'Attack': 90, 'Defense': 50, 'HP': 65, 'Sp. A...","{'ability': [['Chlorophyll', 'false'], ['Glutt...",Weepinbell,"[-0.01772996410727501, -0.036318209022283554, ...",69,0.121085
3,Drowsing Pokémon: It remains asleep from birth...,775,['Normal'],"{'Attack': 115, 'Defense': 65, 'HP': 65, 'Sp. ...","{'ability': [['Comatose', 'false']], 'egg': ['...",Komala,"[0.019138192757964134, -0.026016755029559135, ...",774,0.104583
4,Pitch-Black Pokémon: It can lull people to sle...,491,['Dark'],"{'Attack': 90, 'Defense': 90, 'HP': 70, 'Sp. A...","{'ability': [['Bad Dreams', 'false']], 'egg': ...",Darkrai,"[0.044289879500865936, -0.04118366539478302, 0...",490,0.088791


## Querying the collection's embeddings, eager execution

In [11]:
query = "Loves to sleep"
sleepy_pokemon = collection.query(query={"description":["Loves to sleep"]}, k=10, lazy=False)
sleepy_pokemon_pandas = sleepy_pokemon.to_pandas()
sleepy_pokemon_pandas

,description,id,type,info,profile,name,embeddings_description,modal_map_index,score
0,Sleeping Pokémon: It has no interest in anythi...,143,['Normal'],"{'Attack': 110, 'Defense': 65, 'HP': 160, 'Sp....","{'ability': [['Immunity', 'false'], ['Thick Fa...",Snorlax,"[0.012115702033042908, -0.006682790350168943, ...",142,0.465361
1,Tiny Robin Pokémon: Its body is always warm. T...,661,"['Normal', 'Flying']","{'Attack': 50, 'Defense': 43, 'HP': 45, 'Sp. A...","{'ability': [['Big Pecks', 'false'], ['Gale Wi...",Fletchling,"[0.01935332827270031, -0.08202998340129852, 0....",660,0.418864
2,Drowsing Pokémon: It remains asleep from birth...,775,['Normal'],"{'Attack': 115, 'Defense': 65, 'HP': 65, 'Sp. ...","{'ability': [['Comatose', 'false']], 'egg': ['...",Komala,"[0.019138192757964134, -0.026016755029559135, ...",774,0.411955
3,"Scout Pokémon: When Sentret sleeps, it does so...",161,['Normal'],"{'Attack': 46, 'Defense': 34, 'HP': 35, 'Sp. A...","{'ability': [['Run Away', 'false'], ['Keen Eye...",Sentret,"[-0.007474727928638458, -0.09187264740467072, ...",160,0.410189
4,Prim Pokémon: Delcatty sleeps anywhere it want...,301,['Normal'],"{'Attack': 65, 'Defense': 65, 'HP': 70, 'Sp. A...","{'ability': [['Cute Charm', 'false'], ['Normal...",Delcatty,"[0.012523137964308262, 0.0025400533340871334, ...",300,0.394165
5,Flycatcher Pokémon: Weepinbell has a large hoo...,70,"['Grass', 'Poison']","{'Attack': 90, 'Defense': 50, 'HP': 65, 'Sp. A...","{'ability': [['Chlorophyll', 'false'], ['Glutt...",Weepinbell,"[-0.01772996410727501, -0.036318209022283554, ...",69,0.390755
6,Soft Tissue Pokémon: Its body is mostly water....,704,['Dragon'],"{'Attack': 50, 'Defense': 35, 'HP': 45, 'Sp. A...","{'ability': [['Sap Sipper', 'false'], ['Hydrat...",Goomy,"[0.016569413244724274, -0.023747211322188377, ...",703,0.385355
7,Hypnosis Pokémon: It puts its prey to sleep an...,96,['Psychic'],"{'Attack': 48, 'Defense': 45, 'HP': 60, 'Sp. A...","{'ability': [['Insomnia', 'false'], ['Forewarn...",Drowzee,"[0.06272145360708237, 0.06499218940734863, -0....",95,0.377276
8,Pitch-Black Pokémon: It can lull people to sle...,491,['Dark'],"{'Attack': 90, 'Defense': 90, 'HP': 70, 'Sp. A...","{'ability': [['Bad Dreams', 'false']], 'egg': ...",Darkrai,"[0.044289879500865936, -0.04118366539478302, 0...",490,0.373461
9,"Mouse Pokémon: While sleeping, it generates el...",25,['Electric'],"{'Attack': 55, 'Defense': 40, 'HP': 35, 'Sp. A...","{'ability': [['Static', 'false'], ['Lightning ...",Pikachu,"[0.007011639419943094, -0.03749552369117737, -...",24,0.370107


## Performing filters

### With vexpresso we can leverage duckdb for powerful queries

#### In order to filter on metadata, you must supply a filter dictionary to the filter method. The dictionary must have the following structure:

```python
{
    <field>: {
        <filter_method>: <value>
    },
    <field>: {
        <filter_method>: <value>
    },
}

```

#### Query to only get Pokemon with the name "Snorlax":

In [12]:
filter_condition = {
    "name": {
        "eq":"Snorlax"
    }
}
sleepy_pokemon.filter(filter_condition).to_pandas()

,description,id,type,info,profile,name,embeddings_description,modal_map_index,score
0,Sleeping Pokémon: It has no interest in anythi...,143,['Normal'],"{'Attack': 110, 'Defense': 65, 'HP': 160, 'Sp....","{'ability': [['Immunity', 'false'], ['Thick Fa...",Snorlax,"[0.012115702033042908, -0.006682790350168943, ...",142,0.465361


#### Query to only get Pokemon with the name "Snorlax" or "Komala"

In [13]:
filter_condition = {
    "name": {
        "isin":["Snorlax", "Komala"]
    }
}
sleepy_pokemon.filter(filter_condition).to_pandas()

,description,id,type,info,profile,name,embeddings_description,modal_map_index,score
0,Sleeping Pokémon: It has no interest in anythi...,143,['Normal'],"{'Attack': 110, 'Defense': 65, 'HP': 160, 'Sp....","{'ability': [['Immunity', 'false'], ['Thick Fa...",Snorlax,"[0.012115702033042908, -0.006682790350168943, ...",142,0.465361
1,Drowsing Pokémon: It remains asleep from birth...,775,['Normal'],"{'Attack': 115, 'Defense': 65, 'HP': 65, 'Sp. ...","{'ability': [['Comatose', 'false']], 'egg': ['...",Komala,"[0.019138192757964134, -0.026016755029559135, ...",774,0.411955


#### Query to only get Pokemon with type Normal

In [14]:
filter_condition = {
    "type": {
        "contains":"Normal"
    }
}
sleepy_pokemon.filter(filter_condition).to_pandas()

,description,id,type,info,profile,name,embeddings_description,modal_map_index,score
0,Sleeping Pokémon: It has no interest in anythi...,143,['Normal'],"{'Attack': 110, 'Defense': 65, 'HP': 160, 'Sp....","{'ability': [['Immunity', 'false'], ['Thick Fa...",Snorlax,"[0.012115702033042908, -0.006682790350168943, ...",142,0.465361
1,Tiny Robin Pokémon: Its body is always warm. T...,661,"['Normal', 'Flying']","{'Attack': 50, 'Defense': 43, 'HP': 45, 'Sp. A...","{'ability': [['Big Pecks', 'false'], ['Gale Wi...",Fletchling,"[0.01935332827270031, -0.08202998340129852, 0....",660,0.418864
2,Drowsing Pokémon: It remains asleep from birth...,775,['Normal'],"{'Attack': 115, 'Defense': 65, 'HP': 65, 'Sp. ...","{'ability': [['Comatose', 'false']], 'egg': ['...",Komala,"[0.019138192757964134, -0.026016755029559135, ...",774,0.411955
3,"Scout Pokémon: When Sentret sleeps, it does so...",161,['Normal'],"{'Attack': 46, 'Defense': 34, 'HP': 35, 'Sp. A...","{'ability': [['Run Away', 'false'], ['Keen Eye...",Sentret,"[-0.007474727928638458, -0.09187264740467072, ...",160,0.410189
4,Prim Pokémon: Delcatty sleeps anywhere it want...,301,['Normal'],"{'Attack': 65, 'Defense': 65, 'HP': 70, 'Sp. A...","{'ability': [['Cute Charm', 'false'], ['Normal...",Delcatty,"[0.012523137964308262, 0.0025400533340871334, ...",300,0.394165


## Query + Filter chaining

#### For example, let's filter the above collection for Pokemon with HP >= 35 and Defense > 45

In [15]:
filter_conditions = {"info.HP": {"gte":35}, "info.Defense": {"gt":45}}
sleepy_pokemon = collection.query(query={"description":["Loves to sleep"]}, k=10).filter(filter_conditions=filter_conditions).execute()

In [16]:
sleepy_pokemon.to_pandas()

,description,id,type,info,profile,name,embeddings_description,modal_map_index,score
0,Sleeping Pokémon: It has no interest in anythi...,143,['Normal'],"{'Attack': 110, 'Defense': 65, 'HP': 160, 'Sp....","{'ability': [['Immunity', 'false'], ['Thick Fa...",Snorlax,"[0.012115702033042908, -0.006682790350168943, ...",142,0.465361
1,Drowsing Pokémon: It remains asleep from birth...,775,['Normal'],"{'Attack': 115, 'Defense': 65, 'HP': 65, 'Sp. ...","{'ability': [['Comatose', 'false']], 'egg': ['...",Komala,"[0.019138192757964134, -0.026016755029559135, ...",774,0.411955
2,Prim Pokémon: Delcatty sleeps anywhere it want...,301,['Normal'],"{'Attack': 65, 'Defense': 65, 'HP': 70, 'Sp. A...","{'ability': [['Cute Charm', 'false'], ['Normal...",Delcatty,"[0.012523137964308262, 0.0025400533340871334, ...",300,0.394165
3,Flycatcher Pokémon: Weepinbell has a large hoo...,70,"['Grass', 'Poison']","{'Attack': 90, 'Defense': 50, 'HP': 65, 'Sp. A...","{'ability': [['Chlorophyll', 'false'], ['Glutt...",Weepinbell,"[-0.01772996410727501, -0.036318209022283554, ...",69,0.390755
4,Pitch-Black Pokémon: It can lull people to sle...,491,['Dark'],"{'Attack': 90, 'Defense': 90, 'HP': 70, 'Sp. A...","{'ability': [['Bad Dreams', 'false']], 'egg': ...",Darkrai,"[0.044289879500865936, -0.04118366539478302, 0...",490,0.373461
5,Emotion Pokémon: It sleeps at the bottom of a ...,481,['Psychic'],"{'Attack': 105, 'Defense': 105, 'HP': 80, 'Sp....","{'ability': [['Levitate', 'false']], 'egg': ['...",Mesprit,"[0.03139404579997063, -0.030525697395205498, -...",480,0.353983
6,"Stag Beetle Pokémon: Although it’s tough, it c...",127,['Bug'],"{'Attack': 125, 'Defense': 100, 'HP': 65, 'Sp....","{'ability': [['Hyper Cutter', 'false'], ['Mold...",Pinsir,"[0.015433314256370068, -0.07257808744907379, 0...",126,0.352403
7,Dragon Pokémon: Kingdra sleeps on the seafloor...,230,"['Water', 'Dragon']","{'Attack': 95, 'Defense': 95, 'HP': 75, 'Sp. A...","{'ability': [['Swift Swim', 'false'], ['Sniper...",Kingdra,"[0.036651819944381714, -0.0006486555794253945,...",229,0.337267
8,"Sea Lion Pokémon: Thanks to its thick fat, col...",86,['Water'],"{'Attack': 45, 'Defense': 55, 'HP': 65, 'Sp. A...","{'ability': [['Thick Fat', 'false'], ['Hydrati...",Seel,"[0.024392733350396156, 0.0032551344484090805, ...",85,0.322567
9,Moonlight Pokémon: It lurks in the dark of nig...,197,['Dark'],"{'Attack': 65, 'Defense': 110, 'HP': 95, 'Sp. ...","{'ability': [['Synchronize', 'false'], ['Inner...",Umbreon,"[0.0488753467798233, -0.06575425714254379, -0....",196,0.314591


## Grab fields from metadata

In [17]:
names, types, descriptions = sleepy_pokemon.get_fields(["name", "type", "description"])
for n, t, d in zip(names, types, descriptions):
    print(f"{n}: {t} -- {d}")

Snorlax: ['Normal'] -- Sleeping Pokémon: It has no interest in anything other than eating. Even if you climb up on its stomach while it’s napping, it doesn’t seem to mind at all!
Komala: ['Normal'] -- Drowsing Pokémon: It remains asleep from birth to death as a result of the sedative properties of the leaves that form its diet.
Delcatty: ['Normal'] -- Prim Pokémon: Delcatty sleeps anywhere it wants without keeping a permanent nest. If other Pokémon approach it as it sleeps, this Pokémon will never fight—it will just move away somewhere else.
Weepinbell: ['Grass', 'Poison'] -- Flycatcher Pokémon: Weepinbell has a large hook on its rear end. At night, the Pokémon hooks on to a tree branch and goes to sleep. If it moves around in its sleep, it may wake up to find itself on the ground.
Darkrai: ['Dark'] -- Pitch-Black Pokémon: It can lull people to sleep and make them dream. It is active during nights of the new moon.
Mesprit: ['Psychic'] -- Emotion Pokémon: It sleeps at the bottom of a la

## Save collection

In [18]:
sleepy_pokemon.save("./saved_filtered_pokemon")

saving to ./saved_filtered_pokemon


## Load collection

In [19]:
loaded_collection = vexpresso.create(
    directory_or_repo_id = "saved_filtered_pokemon",
    embedding_fn=HuggingFaceEmbeddings(),
    retriever=NumpyRetriever()
)

In [20]:
loaded_collection.show(5)

descriptionUtf8,idInt64,typeUtf8,"infoStruct[Attack: Int64, Defense: Int64, HP: Int64, Sp. Attack: Int64, Sp. Defense: Int64, Speed: Int64]","profileStruct[ability: List[List[Utf8]], egg: List[Utf8], gender: Utf8, height: Utf8, weight: Utf8]",nameUtf8,embeddings_descriptionPython,modal_map_indexInt64,scoreFloat64
Sleeping Pokémon: It has no interest in anything other th...,143,['Normal'],"{'Attack': 110, 'Defense': 65, 'HP': 160, 'Sp. Attack': 6...","{'ability': [['Immunity', 'false'], ['Thick Fat', 'false'...",Snorlax,"<np.ndarrayshape=(768,)dtype=float64>",142,0.465361
Drowsing Pokémon: It remains asleep from birth to death a...,775,['Normal'],"{'Attack': 115, 'Defense': 65, 'HP': 65, 'Sp. Attack': 75...","{'ability': [['Comatose', 'false']], 'egg': ['Field'], 'g...",Komala,"<np.ndarrayshape=(768,)dtype=float64>",774,0.411955
Prim Pokémon: Delcatty sleeps anywhere it wants without k...,301,['Normal'],"{'Attack': 65, 'Defense': 65, 'HP': 70, 'Sp. Attack': 55,...","{'ability': [['Cute Charm', 'false'], ['Normalize', 'fals...",Delcatty,"<np.ndarrayshape=(768,)dtype=float64>",300,0.394165
Flycatcher Pokémon: Weepinbell has a large hook on its re...,70,"['Grass', 'Poison']","{'Attack': 90, 'Defense': 50, 'HP': 65, 'Sp. Attack': 85,...","{'ability': [['Chlorophyll', 'false'], ['Gluttony', 'true...",Weepinbell,"<np.ndarrayshape=(768,)dtype=float64>",69,0.390755
Pitch-Black Pokémon: It can lull people to sleep and make...,491,['Dark'],"{'Attack': 90, 'Defense': 90, 'HP': 70, 'Sp. Attack': 135...","{'ability': [['Bad Dreams', 'false']], 'egg': ['Undiscove...",Darkrai,"<np.ndarrayshape=(768,)dtype=float64>",490,0.373461


## Uploading to HFhub

In [21]:
# Automatically gets token from env variable
# HUGGINGFACEHUB_API_TOKEN = ...
# or you can pass in token directly via collection.save(token=...)
# repo_id = {HF_USERNAME}/{REPO_NAME}
repo_id = "shyamsn97/pokemon_vexpresso_test"
loaded_collection.save(repo_id, to_hub=True, private=True)

Uploading collection to shyamsn97/pokemon_vexpresso_test
Upload to shyamsn97/pokemon_vexpresso_test complete!


'shyamsn97/pokemon_vexpresso_test'

In [22]:
# or upload with username + repo name
loaded_collection.save(hf_username = "shyamsn97", repo_name = "pokemon_vexpresso_test", to_hub=True)

Uploading collection to None
Upload to shyamsn97/pokemon_vexpresso_test complete!


'shyamsn97/pokemon_vexpresso_test'

## Load from HfHub

In [23]:
# Automatically gets token from env variable
# HUGGINGFACEHUB_API_TOKEN = ...
# or you can pass in token directly via Collection.load(token=...)
# repo_id = {HF_USERNAME}/{REPO_NAME}
repo_id = "shyamsn97/pokemon_vexpresso_test"

collection = vexpresso.create(
    directory_or_repo_id = repo_id,
    embedding_fn=HuggingFaceEmbeddings(),
    retriever=NumpyRetriever()
)
collection.to_pandas()

Retrieving from hf repo: shyamsn97/pokemon_vexpresso_test


Fetching 2 files: 100%|███████████████████████████████████████████████████| 2/2 [00:00<00:00, 10.88it/s]


,description,id,type,info,profile,name,embeddings_description,modal_map_index,score
0,Sleeping Pokémon: It has no interest in anythi...,143,['Normal'],"{'Attack': 110, 'Defense': 65, 'HP': 160, 'Sp....","{'ability': [['Immunity', 'false'], ['Thick Fa...",Snorlax,"[0.012115702033042908, -0.006682790350168943, ...",142,0.465361
1,Drowsing Pokémon: It remains asleep from birth...,775,['Normal'],"{'Attack': 115, 'Defense': 65, 'HP': 65, 'Sp. ...","{'ability': [['Comatose', 'false']], 'egg': ['...",Komala,"[0.019138192757964134, -0.026016755029559135, ...",774,0.411955
2,Prim Pokémon: Delcatty sleeps anywhere it want...,301,['Normal'],"{'Attack': 65, 'Defense': 65, 'HP': 70, 'Sp. A...","{'ability': [['Cute Charm', 'false'], ['Normal...",Delcatty,"[0.012523137964308262, 0.0025400533340871334, ...",300,0.394165
3,Flycatcher Pokémon: Weepinbell has a large hoo...,70,"['Grass', 'Poison']","{'Attack': 90, 'Defense': 50, 'HP': 65, 'Sp. A...","{'ability': [['Chlorophyll', 'false'], ['Glutt...",Weepinbell,"[-0.01772996410727501, -0.036318209022283554, ...",69,0.390755
4,Pitch-Black Pokémon: It can lull people to sle...,491,['Dark'],"{'Attack': 90, 'Defense': 90, 'HP': 70, 'Sp. A...","{'ability': [['Bad Dreams', 'false']], 'egg': ...",Darkrai,"[0.044289879500865936, -0.04118366539478302, 0...",490,0.373461
5,Emotion Pokémon: It sleeps at the bottom of a ...,481,['Psychic'],"{'Attack': 105, 'Defense': 105, 'HP': 80, 'Sp....","{'ability': [['Levitate', 'false']], 'egg': ['...",Mesprit,"[0.03139404579997063, -0.030525697395205498, -...",480,0.353983
6,"Stag Beetle Pokémon: Although it’s tough, it c...",127,['Bug'],"{'Attack': 125, 'Defense': 100, 'HP': 65, 'Sp....","{'ability': [['Hyper Cutter', 'false'], ['Mold...",Pinsir,"[0.015433314256370068, -0.07257808744907379, 0...",126,0.352403
7,Dragon Pokémon: Kingdra sleeps on the seafloor...,230,"['Water', 'Dragon']","{'Attack': 95, 'Defense': 95, 'HP': 75, 'Sp. A...","{'ability': [['Swift Swim', 'false'], ['Sniper...",Kingdra,"[0.036651819944381714, -0.0006486555794253945,...",229,0.337267
8,"Sea Lion Pokémon: Thanks to its thick fat, col...",86,['Water'],"{'Attack': 45, 'Defense': 55, 'HP': 65, 'Sp. A...","{'ability': [['Thick Fat', 'false'], ['Hydrati...",Seel,"[0.024392733350396156, 0.0032551344484090805, ...",85,0.322567
9,Moonlight Pokémon: It lurks in the dark of nig...,197,['Dark'],"{'Attack': 65, 'Defense': 110, 'HP': 95, 'Sp. ...","{'ability': [['Synchronize', 'false'], ['Inner...",Umbreon,"[0.0488753467798233, -0.06575425714254379, -0....",196,0.314591


In [24]:
# or load with username + repo name
collection = vexpresso.create(
    hf_username = "shyamsn97",
    repo_name = "pokemon_vexpresso_test",
    embedding_fn=HuggingFaceEmbeddings(),
    retriever=NumpyRetriever()
)
collection.to_pandas()

Retrieving from hf repo: shyamsn97/pokemon_vexpresso_test


Fetching 2 files: 100%|████████████████████████████████████████████████| 2/2 [00:00<00:00, 14873.42it/s]


,description,id,type,info,profile,name,embeddings_description,modal_map_index,score
0,Sleeping Pokémon: It has no interest in anythi...,143,['Normal'],"{'Attack': 110, 'Defense': 65, 'HP': 160, 'Sp....","{'ability': [['Immunity', 'false'], ['Thick Fa...",Snorlax,"[0.012115702033042908, -0.006682790350168943, ...",142,0.465361
1,Drowsing Pokémon: It remains asleep from birth...,775,['Normal'],"{'Attack': 115, 'Defense': 65, 'HP': 65, 'Sp. ...","{'ability': [['Comatose', 'false']], 'egg': ['...",Komala,"[0.019138192757964134, -0.026016755029559135, ...",774,0.411955
2,Prim Pokémon: Delcatty sleeps anywhere it want...,301,['Normal'],"{'Attack': 65, 'Defense': 65, 'HP': 70, 'Sp. A...","{'ability': [['Cute Charm', 'false'], ['Normal...",Delcatty,"[0.012523137964308262, 0.0025400533340871334, ...",300,0.394165
3,Flycatcher Pokémon: Weepinbell has a large hoo...,70,"['Grass', 'Poison']","{'Attack': 90, 'Defense': 50, 'HP': 65, 'Sp. A...","{'ability': [['Chlorophyll', 'false'], ['Glutt...",Weepinbell,"[-0.01772996410727501, -0.036318209022283554, ...",69,0.390755
4,Pitch-Black Pokémon: It can lull people to sle...,491,['Dark'],"{'Attack': 90, 'Defense': 90, 'HP': 70, 'Sp. A...","{'ability': [['Bad Dreams', 'false']], 'egg': ...",Darkrai,"[0.044289879500865936, -0.04118366539478302, 0...",490,0.373461
5,Emotion Pokémon: It sleeps at the bottom of a ...,481,['Psychic'],"{'Attack': 105, 'Defense': 105, 'HP': 80, 'Sp....","{'ability': [['Levitate', 'false']], 'egg': ['...",Mesprit,"[0.03139404579997063, -0.030525697395205498, -...",480,0.353983
6,"Stag Beetle Pokémon: Although it’s tough, it c...",127,['Bug'],"{'Attack': 125, 'Defense': 100, 'HP': 65, 'Sp....","{'ability': [['Hyper Cutter', 'false'], ['Mold...",Pinsir,"[0.015433314256370068, -0.07257808744907379, 0...",126,0.352403
7,Dragon Pokémon: Kingdra sleeps on the seafloor...,230,"['Water', 'Dragon']","{'Attack': 95, 'Defense': 95, 'HP': 75, 'Sp. A...","{'ability': [['Swift Swim', 'false'], ['Sniper...",Kingdra,"[0.036651819944381714, -0.0006486555794253945,...",229,0.337267
8,"Sea Lion Pokémon: Thanks to its thick fat, col...",86,['Water'],"{'Attack': 45, 'Defense': 55, 'HP': 65, 'Sp. A...","{'ability': [['Thick Fat', 'false'], ['Hydrati...",Seel,"[0.024392733350396156, 0.0032551344484090805, ...",85,0.322567
9,Moonlight Pokémon: It lurks in the dark of nig...,197,['Dark'],"{'Attack': 65, 'Defense': 110, 'HP': 95, 'Sp. ...","{'ability': [['Synchronize', 'false'], ['Inner...",Umbreon,"[0.0488753467798233, -0.06575425714254379, -0....",196,0.314591
